In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc
from datetime import datetime
from rapidfuzz import process, fuzz
from Levenshtein import distance

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')
actas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v2/reportes_actas.csv')
personas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v3/reporte_personas.csv')

In [ ]:
cbc['Dni'] = cbc['Dni'].astype(str)
actas['fecha'] = pd.to_datetime(actas['fecha'], format='%Y-%m-%d')
actas['año'] = actas['fecha'].dt.year
actas['mes'] = actas['fecha'].dt.month

Se que todas las actas se conectan con alguna persona, el problema es que hay muchos de las actas que no se conectan con el cbc, intentamos hacer un join que no sea por exactitud, que sea por 'like' y revisamos a mano los casos donde no sean exactamente iguales.

In [ ]:
cbc.columns

In [ ]:
actas['dni'].nunique()

Chequeo que todos los DNIs del CBC tienen aunque sea una materia de la FCEN.

In [ ]:
# Crear una columna booleana para saber si es materia FCEN
cbc['es_fcen_binario'] = cbc['Es materia FCEN?'].str.upper() == 'SI'

# Agrupar por DNI y verificar si hay al menos un True
resultado_cbc = cbc.groupby('Dni')['es_fcen_binario'].any().reset_index()

In [ ]:
resultado_cbc

In [ ]:
resultado_cbc['es_fcen_binario'].sum()

In [ ]:
dni_sin_fcen = resultado_cbc[~resultado_cbc['es_fcen_binario']]

In [ ]:
dni_sin_fcen['Dni'].nunique()

In [ ]:
cbc[cbc['Dni'].isin(dni_sin_fcen['Dni'])]

In [ ]:
actas[actas['dni'].isin(dni_sin_fcen['Dni'])]['dni'].nunique()

In [ ]:
actas[actas['dni'] == '43447240']

In [ ]:
personas[personas['dni'].isin(dni_sin_fcen['Dni'])][['dni', 'año_inscripcion_facultad']].sort_values(by='dni')

In [ ]:
cbc[cbc['Dni'].isin(actas[actas['dni'].isin(dni_sin_fcen['Dni'])]['dni'].unique())].sort_values('Dni')

Hay un par de DNIs que no tienen registro de rendir materias de la FCEN, para los que se encuentran en las actas logramos ver que en las materias de las que tenemos registro se corresponden con materias de otras carreras que hicieron posteriormente a haberse anotado a una carrera de la FCEN.

Lo que buscamos es encontrarle un par a los DNIs que figuran en las actas/personas de la FCEN que no tienen asignado uno del CBC, por lo tanto:

- Nos quedamos con los DNIs del CBC que estan 'solos' (no se le asigno un DNI exactamente igual)

- Nos guardanos la fecha del ultimo examen rendido en el CBC de una materia de la FCEN

- Nos fijamos que el DNI que se le asigna como mas similar tenga la ultima materia rendida en el mismo año o antes del año de inscripción a la FCEN

In [ ]:
# Me quedo con datos que no se repiten del cbc
cbc['anio_rendido'] = cbc['Fecha'].dt.year
cbc_solo_dni = cbc['Dni'].unique()
cbc_dnis = cbc[['Dni']].drop_duplicates()
cbc_con_fcen = cbc[cbc['es_fcen_binario']]
df_ultimos_rendidos = cbc_con_fcen.groupby('Dni', as_index=False)['anio_rendido'].max()
df_ultimos_rendidos.rename(columns={'anio_rendido': 'ultimo_anio_rendido'}, inplace=True)

# Paso 4: Hacer el join para incluir todos los DNIs
df_resultado = cbc_dnis.merge(df_ultimos_rendidos, on='Dni', how='left')

In [ ]:
df_resultado.replace({'ultimo_anio_rendido': {np.nan: 2019}}, inplace=True)

In [ ]:
actas_solo_dni = actas[~actas['dni'].isin(cbc_solo_dni)]['dni'].unique()
cbc_solo_dni_sin_par = df_resultado[~df_resultado['Dni'].isin(actas['dni'])]['Dni'].unique()

In [ ]:
actas_solo_dni.shape, cbc_solo_dni_sin_par.shape

Usamos esto si no consideramos que tenemos el dato del año

In [ ]:
# Función para hacer fuzzy match y obtener el DNI más parecido y el score
def fuzzy_match_with_score(dni, choices):
    match = process.extractOne(dni, choices, scorer=fuzz.ratio)
    if match:
        return pd.Series([match[0], match[1]])  # DNI más parecido y score
    return pd.Series([None, None])

# Aplicamos la función a cada DNI
df_resultado = pd.DataFrame({'dni_original_actas': actas_solo_dni})
df_resultado[['dni_match', 'similarity_score']] = df_resultado['dni_original_actas'].apply(
    lambda x: fuzzy_match_with_score(x, cbc_solo_dni_sin_par)
)

# Ordenamos para ver los peores matches primero
df_resultado = df_resultado.sort_values(by='similarity_score', ascending=False)

print(df_resultado)

Vale que si el score es 100 es porque son exatamente iguales, debería revisar lo que no son iguales a 100

In [ ]:
df_resultado[df_resultado['similarity_score'] == 100].shape

In [ ]:
(df_resultado[df_resultado['similarity_score'] == 100]['dni_original_actas'] == df_resultado[df_resultado['similarity_score'] == 100]['dni_match']).sum()

In [ ]:
df_resultado[df_resultado['similarity_score'] != 100].to_csv('dni_fuzzy_match_distinto_100.csv', index=False)

Pero si consideramos que tenemos el datos del año

In [ ]:
# Tiene dato del CBC
df_resultado = df_resultado[~df_resultado['Dni'].isin(actas['dni'])]
df_resultado.head(10)

In [ ]:
personas_match = personas[~personas['dni'].isin(cbc_solo_dni)][['dni', 'año_inscripcion_facultad']].sort_values(by='dni')

In [ ]:
# Resultado final
resultados_match = []

for _, row in personas_match.iterrows():
    dni_nuevo = row['dni']
    anio_inscripcion = row['año_inscripcion_facultad']
    
    # Filtrar base contra la cual matchear
    candidatos = df_resultado[df_resultado['ultimo_anio_rendido'] < anio_inscripcion]
    
    # Aplicar fuzzy matching
    match = process.extractOne(
        dni_nuevo,
        candidatos['Dni'],
        scorer=fuzz.ratio
    )
    
    if match:
        dni_matcheado, score, _ = match
        resultados_match.append({
            'dni_nuevo': dni_nuevo,
            'anio_inscripcion': anio_inscripcion,
            'dni_matcheado': dni_matcheado,
            'similitud': score
        })
    else:
        resultados_match.append({
            'dni_nuevo': dni_nuevo,
            'anio_inscripcion': anio_inscripcion,
            'dni_matcheado': None,
            'similitud': None
        })

# Pasar a DataFrame
df_matches = pd.DataFrame(resultados_match)

print(df_matches)

Probamos esta otra forma que se fija que se diferencien solamente en un caracter.

In [ ]:
# Crear un DataFrame para almacenar los resultados
resultados = []
dnis_1 = actas_solo_dni
dnis_2 = cbc_solo_dni_sin_par
for dni1 in dnis_1:
    for dni2 in dnis_2:
        dist = distance(dni1, dni2)
        if dist == 1:
            resultados.append({'actas': dni1, 'cbc': dni2, 'diferencia': dist})

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

In [ ]:
df_resultados.to_csv('dni_levenshtein_dist_1.csv', index=False)

Un tema es que no es unico el match, habría que revisar eso, en caso de que tenga real sentido unir dos mismos dnis

Miramos qué pasa si unimos mirando también que los años tengan sentido

In [ ]:
resultados_match = []

for _, row in personas_match.iterrows():
    dni_nuevo = row['dni']
    anio_inscripcion = row['año_inscripcion_facultad']
    
    # Filtrar candidatos válidos
    candidatos = df_resultado[df_resultado['ultimo_anio_rendido'] < anio_inscripcion]
    
    # Extraer lista de DNIs candidatos
    lista_candidatos = candidatos['Dni'].tolist()
    
    if lista_candidatos:
        match = process.extractOne(
            dni_nuevo,
            lista_candidatos,
            scorer=distance.Levenshtein.distance  # menor es mejor
        )
        
        if match:
            dni_matcheado, distancia, _ = match
            resultados_match.append({
                'dni_nuevo': dni_nuevo,
                'anio_inscripcion': anio_inscripcion,
                'dni_matcheado': dni_matcheado,
                'levenshtein_distancia': distancia
            })
    else:
        resultados_match.append({
            'dni_nuevo': dni_nuevo,
            'anio_inscripcion': anio_inscripcion,
            'dni_matcheado': None,
            'levenshtein_distancia': None
        })

# Convertir en DataFrame
df_matches = pd.DataFrame(resultados_match)

# (Opcional) Unir con df_resultado para traer el último año de la facultad del dni_matcheado
df_matches = df_matches.merge(
    df_resultado.rename(columns={'Dni': 'dni_matcheado'}),
    on='dni_matcheado',
    how='left'
)

print(df_matches)

In [ ]:
df_matches['levenshtein_distancia'].isnull().sum()

In [ ]:
df_matches['levenshtein_distancia'].value_counts()

Voy a mirar los casos donde la distiancia es de un caracter

In [ ]:
df_matches[df_matches['levenshtein_distancia'] == 1].to_csv('dni_levenshtein_dist_1_considerando_anio.csv', index=False)

In [ ]:
df_matches[df_matches['levenshtein_distancia'] == 1]['dni_matcheado'].nunique()

In [ ]:
df_matches[df_matches['anio_inscripcion'] == 2022].sort_values('levenshtein_distancia', ascending=True).to_csv('dni_levenshtein_dist_1_considerando_anio_2022.csv', index=False)

In [ ]:
cbc_distancia_1 = cbc[cbc['Dni'].isin(df_matches[(df_matches['anio_inscripcion'] == 2022) & (df_matches['levenshtein_distancia'] == 1)]['dni_matcheado'])]

In [ ]:
personas_distancia_1 = personas[personas['dni'].isin(df_matches[(df_matches['anio_inscripcion'] == 2022) & (df_matches['levenshtein_distancia'] == 1)]['dni_nuevo'])]